##### https://wooiljeong.github.io/python/pdr-code/

In [1]:
import PublicDataReader as pdr
import pandas as pd

In [2]:
sigungu_code = pdr.code_bdong()
sigungu_code.head()

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.8.1. 시행)


,시도코드,시도명,시군구코드,시군구명,법정동코드,읍면동명,동리명,생성일자,말소일자
0,11,서울특별시,11000,,1100000000,,,19880423,
1,11,서울특별시,11110,종로구,1111000000,,,19880423,
2,11,서울특별시,11110,종로구,1111010100,청운동,,19880423,
3,11,서울특별시,11110,종로구,1111010200,신교동,,19880423,
4,11,서울특별시,11110,종로구,1111010300,궁정동,,19880423,


In [3]:
# 국토교통부_아파트 분양권전매 실거래가 자료 Decoding Key
service_key = "LvPC1qYZfxGa3thN77tc4RrTq9wofgs/fCpwE/Bf6SILskFce1Uwuu4xea2bt8dpybj9LekLlLK7WpRvo4JcaQ=="

In [4]:
from PublicDataReader import TransactionPrice
api_right = TransactionPrice(service_key)

In [5]:
sigungu_codes = sigungu_code['시군구코드'].unique().tolist() # 시군구 Code List 생성

In [6]:
start_year_month = 200706
end_year_month = 202507

In [ ]:
import pandas as pd

# 전국 전체 기간 분양권/입주권 데이터 가져오기
all_right_data_list = [] # 분양권/입주권 데이터를 저장할 빈 리스트 생성

# 데이터 수집 및 합치기
for sigungu_code_item in sigungu_codes:
    try:
        print(f"Fetching data for sigungu code: {sigungu_code_item}")
        # 분양입주권 데이터 가져오기
        right_data = api_right.get_data(
            property_type="분양입주권",
            trade_type="매매", # 거래 유형은 매매로 설정
            sigungu_code=sigungu_code_item,
            start_year_month=start_year_month, # start_year_month 변수 사용
            end_year_month=end_year_month, # end_year_month 변수 사용
        )

        # 시도명/시군구명 컬럼 추가
        right_data = pd.merge(
            right_data,
            sigungu_code[['시도명', '시군구명', '시군구코드']].drop_duplicates(subset=['시군구코드']),
            left_on='법정동시군구코드',
            right_on='시군구코드',
            how='left'
        )

        # 병합에 사용된 임시 컬럼 제거
        right_data = right_data.drop(columns=['시군구코드'])

        all_right_data_list.append(right_data)

    except Exception as e:
        print(f"Error fetching data for sigungu code {sigungu_code_item}: {e}")

# 최종 Data Frame 생성
combined_right_data = pd.concat(all_right_data_list, ignore_index=True) # Data Frame 병합

# '구분' 컬럼 값 변경: '입'이면 '입주권', 'None'이면 '분양권'으로 변경
if '구분' in combined_right_data.columns:
    combined_right_data.loc[combined_right_data['구분'] == '입', '구분'] = '입주권'  
    combined_right_data.loc[combined_right_data['구분'] == 'None', '구분'] = '분양권'


# '리' 컬럼 생성 및 초기화 (필요시)
if '법정동' in combined_right_data.columns:
    combined_right_data['리'] = ''

    # '법정동' 컬럼 값을 공백으로 분리하여 '법정동'과 '리' 컬럼에 할당
    def split_법정동(row):
        if isinstance(row['법정동'], str): # 법정동이 문자열인 경우에만 분리 시도
            parts = row['법정동'].split()
            if len(parts) == 2:
                row['법정동'] = parts[0]
                row['리'] = parts[1]
        return row

    combined_right_data = combined_right_data.apply(split_법정동, axis=1)

# '계약년도', '계약월', '계약일' 컬럼을 합쳐 '계약일자' 컬럼 생성 및 날짜 타입으로 변환
if all(col in combined_right_data.columns for col in ['계약년도', '계약월', '계약일']):
    combined_right_data['계약일자'] = pd.to_datetime(
        combined_right_data['계약년도'].astype(str) + '-' +
        combined_right_data['계약월'].astype(str) + '-' +
        combined_right_data['계약일'].astype(str),
        errors='coerce' # 날짜 변환 중 오류 발생 시 NaT로 처리
    )

# 컬럼 이름 변경
combined_right_data = combined_right_data.rename(columns={
    '시도명': '시도',
    '시군구명': '시군구',
    '단지명': '단지'
})

display(combined_right_data.head())